In [1]:
# Copyright (c) 2015-present, Facebook, Inc.
# All rights reserved.
import argparse
import datetime
import numpy as np
import time
import torch
import torch.backends.cudnn as cudnn
import json

from pathlib import Path

from timm.data import Mixup
from timm.models import create_model
from timm.loss import LabelSmoothingCrossEntropy, SoftTargetCrossEntropy
from timm.scheduler import create_scheduler
from timm.optim import create_optimizer
from timm.utils import NativeScaler, get_state_dict, ModelEma

from datasets import build_dataset
from engine import train_one_epoch, evaluate
import SwinTransformer
import utils
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

import warnings
warnings.filterwarnings('ignore')

In [2]:
def get_args_parser():
    parser = argparse.ArgumentParser('Training Swin Transformer', add_help=False)
    parser.add_argument('--batch-size', default=256, type=int)
    parser.add_argument('--epochs', default=300, type=int)

    # Model parameters
    parser.add_argument('--model', default='Swin_T', type=str, metavar='MODEL',
                        help='Name of model to train')
    parser.add_argument('--input-size', default=224, type=int, help='images input size')

    parser.add_argument('--drop-path', type=float, default=0.2, metavar='PCT',
                        help='Drop path rate (default: 0.2)')

    # Optimizer parameters
    parser.add_argument('--opt', default='adamw', type=str, metavar='OPTIMIZER',
                        help='Optimizer (default: "adamw"')
    parser.add_argument('--opt-eps', default=1e-8, type=float, metavar='EPSILON',
                        help='Optimizer Epsilon (default: 1e-8)')
    parser.add_argument('--opt-betas', default=None, type=float, nargs='+', metavar='BETA',
                        help='Optimizer Betas (default: None, use opt default)')
    parser.add_argument('--clip-grad', type=float, default=5.0, metavar='NORM',
                        help='Clip gradient norm (default: None, no clipping)')
    parser.add_argument('--momentum', type=float, default=0.9, metavar='M',
                        help='SGD momentum (default: 0.9)')
    parser.add_argument('--weight-decay', type=float, default=0.05,
                        help='weight decay (default: 0.05)')

    # Learning rate schedule parameters
    parser.add_argument('--sched', default='cosine', type=str, metavar='SCHEDULER',
                        help='LR scheduler (default: "cosine"')
    parser.add_argument('--lr', type=float, default=5e-4, metavar='LR',
                        help='learning rate (default: 5e-4)')
    parser.add_argument('--lr-noise', type=float, nargs='+', default=None, metavar='pct, pct',
                        help='learning rate noise on/off epoch percentages')
    parser.add_argument('--lr-noise-pct', type=float, default=0.67, metavar='PERCENT',
                        help='learning rate noise limit percent (default: 0.67)')
    parser.add_argument('--lr-noise-std', type=float, default=1.0, metavar='STDDEV',
                        help='learning rate noise std-dev (default: 1.0)')
    parser.add_argument('--warmup-lr', type=float, default=5e-7, metavar='LR',
                        help='warmup learning rate (default: 5e-7)')
    parser.add_argument('--min-lr', type=float, default=5e-6, metavar='LR',
                        help='lower lr bound for cyclic schedulers that hit 0 (1e-5)')

    parser.add_argument('--decay-epochs', type=float, default=30, metavar='N',
                        help='epoch interval to decay LR')
    parser.add_argument('--warmup-epochs', type=int, default=20, metavar='N',
                        help='epochs to warmup LR, if scheduler supports')
    parser.add_argument('--cooldown-epochs', type=int, default=10, metavar='N',
                        help='epochs to cooldown LR at min_lr, after cyclic schedule ends')
    parser.add_argument('--patience-epochs', type=int, default=10, metavar='N',
                        help='patience epochs for Plateau LR scheduler (default: 10')
    parser.add_argument('--decay-rate', '--dr', type=float, default=0.1, metavar='RATE',
                        help='LR decay rate (default: 0.1)')

    # Augmentation parameters
    parser.add_argument('--color-jitter', type=float, default=0.4, metavar='PCT',
                        help='Color jitter factor (default: 0.4)')
    parser.add_argument('--aa', type=str, default='rand-m9-mstd0.5-inc1', metavar='NAME',
                        help='Use AutoAugment policy. "v0" or "original". " + \
                             "(default: rand-m9-mstd0.5-inc1)'),
    parser.add_argument('--smoothing', type=float, default=0.1, help='Label smoothing (default: 0.1)')
    parser.add_argument('--train-interpolation', type=str, default='bicubic',
                        help='Training interpolation (random, bilinear, bicubic default: "bicubic")')

    # * Random Erase params
    parser.add_argument('--reprob', type=float, default=0.25, metavar='PCT',
                        help='Random erase prob (default: 0.25)')
    parser.add_argument('--remode', type=str, default='pixel',
                        help='Random erase mode (default: "pixel")')
    parser.add_argument('--recount', type=int, default=1,
                        help='Random erase count (default: 1)')
    parser.add_argument('--resplit', action='store_true', default=False,
                        help='Do not random erase first (clean) augmentation split')

    # * Mixup params
    parser.add_argument('--mixup', type=float, default=0.8,
                        help='mixup alpha, mixup enabled if > 0. (default: 0.8)')
    parser.add_argument('--cutmix', type=float, default=1.0,
                        help='cutmix alpha, cutmix enabled if > 0. (default: 1.0)')
    parser.add_argument('--cutmix-minmax', type=float, nargs='+', default=None,
                        help='cutmix min/max ratio, overrides alpha and enables cutmix if set (default: None)')
    parser.add_argument('--mixup-prob', type=float, default=1.0,
                        help='Probability of performing mixup or cutmix when either/both is enabled')
    parser.add_argument('--mixup-switch-prob', type=float, default=0.5,
                        help='Probability of switching to cutmix when both mixup and cutmix enabled')
    parser.add_argument('--mixup-mode', type=str, default='batch',
                        help='How to apply mixup/cutmix params. Per "batch", "pair", or "elem"')

    # Dataset parameters
    parser.add_argument('--data-path', default='/datasets01/imagenet_full_size/061417/', type=str,
                        help='dataset path')
    parser.add_argument('--data-set', default='IMNET', choices=['CIFAR', 'IMNET', 'INAT', 'INAT19'],
                        type=str, help='Image Net dataset path')

    parser.add_argument('--output_dir', default='',
                        help='path where to save, empty for no saving')
    parser.add_argument('--device', default='cuda',
                        help='device to use for training / testing')
    parser.add_argument('--seed', default=0, type=int)
    parser.add_argument('--resume', default='', help='resume from checkpoint')
    parser.add_argument('--start_epoch', default=0, type=int, metavar='N',
                        help='start epoch')
    parser.add_argument('--eval', action='store_true', help='Perform evaluation only')
    parser.add_argument('--dist-eval', action='store_true', default=False, help='Enabling distributed evaluation')
    parser.add_argument('--num_workers', default=10, type=int)
    parser.add_argument('--pin-mem', action='store_true',
                        help='Pin CPU memory in DataLoader for more efficient (sometimes) transfer to GPU.')
    parser.add_argument('--no-pin-mem', action='store_false', dest='pin_mem',
                        help='')
    parser.set_defaults(pin_mem=True)

    # distributed training parameters
    parser.add_argument('--world_size', default=1, type=int,
                        help='number of distributed processes')
    parser.add_argument('--dist_url', default='env://', help='url used to set up distributed training')
    return parser

In [3]:
def main(args):
    utils.init_distributed_mode(args)

    print(args)

    device = torch.device(args.device)

    # fix the seed for reproducibility
    seed = args.seed + utils.get_rank()
    torch.manual_seed(seed)
    np.random.seed(seed)
    # random.seed(seed)

    cudnn.benchmark = True

    dataset_train, args.nb_classes = build_dataset(is_train=True, args=args)
    dataset_val, _ = build_dataset(is_train=False, args=args)

    if True:  # args.distributed:
        num_tasks = utils.get_world_size()
        global_rank = utils.get_rank()
        sampler_train = torch.utils.data.DistributedSampler(
            dataset_train, num_replicas=num_tasks, rank=global_rank, shuffle=True
        )
        if args.dist_eval:
            if len(dataset_val) % num_tasks != 0:
                print('Warning: Enabling distributed evaluation with an eval dataset not divisible by process number. '
                      'This will slightly alter validation results as extra duplicate entries are added to achieve '
                      'equal num of samples per-process.')
            sampler_val = torch.utils.data.DistributedSampler(
                dataset_val, num_replicas=num_tasks, rank=global_rank, shuffle=False)
        else:
            sampler_val = torch.utils.data.SequentialSampler(dataset_val)
    else:
        sampler_train = torch.utils.data.RandomSampler(dataset_train)
        sampler_val = torch.utils.data.SequentialSampler(dataset_val)

    data_loader_train = torch.utils.data.DataLoader(
        dataset_train, sampler=sampler_train,
        batch_size=args.batch_size,
        num_workers=args.num_workers,
        pin_memory=args.pin_mem,
        drop_last=True,
    )

    data_loader_val = torch.utils.data.DataLoader(
        dataset_val, sampler=sampler_val,
        batch_size=int(1.0 * args.batch_size),
        num_workers=args.num_workers,
        pin_memory=args.pin_mem,
        drop_last=False
    )

    mixup_fn = None
    mixup_active = args.mixup > 0 or args.cutmix > 0. or args.cutmix_minmax is not None
    if mixup_active:
        mixup_fn = Mixup(
            mixup_alpha=args.mixup, cutmix_alpha=args.cutmix, cutmix_minmax=args.cutmix_minmax,
            prob=args.mixup_prob, switch_prob=args.mixup_switch_prob, mode=args.mixup_mode,
            label_smoothing=args.smoothing, num_classes=args.nb_classes)

    print(f"Creating model: {args.model}")
    # model = create_model(
    #     args.model,
    #     pretrained=False,
    #     num_classes=args.nb_classes,
    #     drop_rate=args.drop,
    #     drop_path_rate=args.drop_path,
    #     drop_block_rate=None,
    # )
    model = getattr(SwinTransformer, args.model)(num_classes=args.nb_classes, drop_path_rate=args.drop_path)
    model.to(device)

    model_without_ddp = model
    if args.distributed:
        model = torch.nn.parallel.DistributedDataParallel(model, device_ids=[args.gpu])
        model_without_ddp = model.module
    n_parameters = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print('number of params:', n_parameters)

    linear_scaled_lr = args.lr * args.batch_size * utils.get_world_size() / 512.0
    args.lr = linear_scaled_lr

    linear_scaled_warmup_lr = args.warmup_lr * args.batch_size * utils.get_world_size() / 512.0
    args.warmup_lr = linear_scaled_warmup_lr

    optimizer = create_optimizer(args, model_without_ddp)
    loss_scaler = NativeScaler()

    lr_scheduler, _ = create_scheduler(args, optimizer)

    # criterion = LabelSmoothingCrossEntropy()

    if args.mixup > 0.:
        # smoothing is handled with mixup label transform
        criterion = SoftTargetCrossEntropy()
    elif args.smoothing:
        criterion = LabelSmoothingCrossEntropy(smoothing=args.smoothing)
    else:
        criterion = torch.nn.CrossEntropyLoss()


    output_dir = Path(args.output_dir)
    if args.resume:
        if args.resume.startswith('https'):
            checkpoint = torch.hub.load_state_dict_from_url(
                args.resume, map_location='cpu', check_hash=True)
        else:
            checkpoint = torch.load(args.resume, map_location='cpu')
        model_without_ddp.load_state_dict(checkpoint['model'])
        if not args.eval and 'optimizer' in checkpoint and 'lr_scheduler' in checkpoint and 'epoch' in checkpoint:
            optimizer.load_state_dict(checkpoint['optimizer'])
            lr_scheduler.load_state_dict(checkpoint['lr_scheduler'])
            args.start_epoch = checkpoint['epoch'] + 1
            if 'scaler' in checkpoint:
                loss_scaler.load_state_dict(checkpoint['scaler'])

    if args.eval:
        test_stats = evaluate(data_loader_val, model, device)
        print(f"Accuracy of the network on the {len(dataset_val)} test images: {test_stats['acc1']:.1f}%")
        return

    print(f"Start training for {args.epochs} epochs")
    start_time = time.time()
    max_accuracy = 0.0
    for epoch in range(args.start_epoch, args.epochs):
        if args.distributed:
            data_loader_train.sampler.set_epoch(epoch)

        lr_scheduler.step(epoch+1)
        train_stats = train_one_epoch(
            model, criterion, data_loader_train,
            optimizer, device, epoch, loss_scaler,
            args.clip_grad, mixup_fn,
            set_training_mode=True  # keep in eval mode during finetuning
        )

        if args.output_dir:
            checkpoint_paths = [output_dir / 'checkpoint.pth']
            for checkpoint_path in checkpoint_paths:
                utils.save_on_master({
                    'model': model_without_ddp.state_dict(),
                    'optimizer': optimizer.state_dict(),
                    'lr_scheduler': lr_scheduler.state_dict(),
                    'epoch': epoch,
                    'scaler': loss_scaler.state_dict(),
                    'args': args,
                }, checkpoint_path)

        test_stats = evaluate(data_loader_val, model, device)
        print(f"Accuracy of the network on the {len(dataset_val)} test images: {test_stats['acc1']:.1f}%")
        max_accuracy = max(max_accuracy, test_stats["acc1"])
        print(f'Max accuracy: {max_accuracy:.2f}%')

        log_stats = {**{f'train_{k}': v for k, v in train_stats.items()},
                     **{f'test_{k}': v for k, v in test_stats.items()},
                     'epoch': epoch,
                     'n_parameters': n_parameters}

        if args.output_dir and utils.is_main_process():
            with (output_dir / "log.txt").open("a") as f:
                f.write(json.dumps(log_stats) + "\n")

    total_time = time.time() - start_time
    total_time_str = str(datetime.timedelta(seconds=int(total_time)))
    print('Training time {}'.format(total_time_str))

In [4]:
parser = argparse.ArgumentParser('Swin Transformer', parents=[get_args_parser()])

In [5]:
args = parser.parse_args(['--model=Swin_T', '--num_workers=4', '--batch-size=64', '--epochs=100', '--drop-path=0.2', '--data-path=data/ILSVRC2012/', '--output_dir=data/SwinT/'])

In [6]:
if args.output_dir:
        Path(args.output_dir).mkdir(parents=True, exist_ok=True)

In [7]:
main(args)

Not using distributed mode
Namespace(batch_size=64, epochs=100, model='Swin_T', input_size=224, drop_path=0.2, opt='adamw', opt_eps=1e-08, opt_betas=None, clip_grad=5.0, momentum=0.9, weight_decay=0.05, sched='cosine', lr=0.0005, lr_noise=None, lr_noise_pct=0.67, lr_noise_std=1.0, warmup_lr=5e-07, min_lr=5e-06, decay_epochs=30, warmup_epochs=20, cooldown_epochs=10, patience_epochs=10, decay_rate=0.1, color_jitter=0.4, aa='rand-m9-mstd0.5-inc1', smoothing=0.1, train_interpolation='bicubic', reprob=0.25, remode='pixel', recount=1, resplit=False, mixup=0.8, cutmix=1.0, cutmix_minmax=None, mixup_prob=1.0, mixup_switch_prob=0.5, mixup_mode='batch', data_path='data/ILSVRC2012/', data_set='IMNET', output_dir='data/SwinT/', device='cuda', seed=0, resume='', start_epoch=0, eval=False, dist_eval=False, num_workers=4, pin_mem=True, world_size=1, dist_url='env://', distributed=False)
Creating model: Swin_T
Block Initial Type: W, drop_path_rate:0.000000
Block Initial Type: SW, drop_path_rate:0.0181

Test:  [280/782]  eta: 0:01:01  loss: 6.6912 (6.5339)  acc1: 0.0000 (0.5449)  acc5: 0.0000 (3.5309)  time: 0.1133  data: 0.0483  max mem: 4817
Test:  [290/782]  eta: 0:00:59  loss: 6.6712 (6.5366)  acc1: 0.0000 (0.5262)  acc5: 0.0000 (3.4418)  time: 0.1131  data: 0.0473  max mem: 4817
Test:  [300/782]  eta: 0:00:58  loss: 6.5873 (6.5344)  acc1: 0.0000 (0.5347)  acc5: 0.0000 (3.5039)  time: 0.1286  data: 0.0633  max mem: 4817
Test:  [310/782]  eta: 0:00:57  loss: 6.4777 (6.5350)  acc1: 0.0000 (0.5778)  acc5: 0.0000 (3.5772)  time: 0.1322  data: 0.0674  max mem: 4817
Test:  [320/782]  eta: 0:00:56  loss: 6.4240 (6.5307)  acc1: 0.0000 (0.6036)  acc5: 1.5625 (3.6458)  time: 0.1077  data: 0.0422  max mem: 4817
Test:  [330/782]  eta: 0:00:54  loss: 6.5552 (6.5341)  acc1: 0.0000 (0.5853)  acc5: 0.0000 (3.5498)  time: 0.1155  data: 0.0498  max mem: 4817
Test:  [340/782]  eta: 0:00:53  loss: 6.6985 (6.5396)  acc1: 0.0000 (0.5682)  acc5: 0.0000 (3.4457)  time: 0.1117  data: 0.0453  max mem: 4817

Epoch: [1]  [ 6000/20018]  eta: 0:42:21  lr: 0.000006  loss: 6.6659 (6.7421)  time: 0.1814  data: 0.0001  max mem: 4818
Epoch: [1]  [ 7000/20018]  eta: 0:39:19  lr: 0.000006  loss: 6.6748 (6.7372)  time: 0.1810  data: 0.0001  max mem: 4818
Epoch: [1]  [ 8000/20018]  eta: 0:36:18  lr: 0.000006  loss: 6.6842 (6.7316)  time: 0.1812  data: 0.0001  max mem: 4818
Epoch: [1]  [ 9000/20018]  eta: 0:33:16  lr: 0.000006  loss: 6.6679 (6.7262)  time: 0.1812  data: 0.0001  max mem: 4818
Epoch: [1]  [10000/20018]  eta: 0:30:15  lr: 0.000006  loss: 6.6752 (6.7212)  time: 0.1810  data: 0.0001  max mem: 4818
Epoch: [1]  [11000/20018]  eta: 0:27:13  lr: 0.000006  loss: 6.6733 (6.7167)  time: 0.1808  data: 0.0001  max mem: 4818
Epoch: [1]  [12000/20018]  eta: 0:24:12  lr: 0.000006  loss: 6.6828 (6.7124)  time: 0.1799  data: 0.0001  max mem: 4818
Epoch: [1]  [13000/20018]  eta: 0:21:10  lr: 0.000006  loss: 6.6756 (6.7078)  time: 0.1801  data: 0.0001  max mem: 4818
Epoch: [1]  [14000/20018]  eta: 0:18:09 

Test:  [440/782]  eta: 0:00:40  loss: 5.9671 (6.0823)  acc1: 0.0000 (2.4589)  acc5: 1.5625 (9.3360)  time: 0.1144  data: 0.0497  max mem: 4818
Test:  [450/782]  eta: 0:00:38  loss: 6.0289 (6.0812)  acc1: 0.0000 (2.4460)  acc5: 4.6875 (9.2849)  time: 0.1212  data: 0.0567  max mem: 4818
Test:  [460/782]  eta: 0:00:37  loss: 6.1602 (6.0829)  acc1: 0.0000 (2.4573)  acc5: 7.8125 (9.2801)  time: 0.1143  data: 0.0499  max mem: 4818
Test:  [470/782]  eta: 0:00:36  loss: 6.3499 (6.0898)  acc1: 0.0000 (2.4151)  acc5: 3.1250 (9.2158)  time: 0.1238  data: 0.0594  max mem: 4818
Test:  [480/782]  eta: 0:00:35  loss: 6.3226 (6.0873)  acc1: 0.0000 (2.4916)  acc5: 1.5625 (9.3295)  time: 0.1140  data: 0.0500  max mem: 4818
Test:  [490/782]  eta: 0:00:34  loss: 6.1056 (6.0875)  acc1: 0.0000 (2.5554)  acc5: 4.6875 (9.4418)  time: 0.1007  data: 0.0371  max mem: 4818
Test:  [500/782]  eta: 0:00:32  loss: 6.1248 (6.0877)  acc1: 3.1250 (2.6260)  acc5: 9.3750 (9.6089)  time: 0.1087  data: 0.0448  max mem: 4818

Test:  [ 10/782]  eta: 0:02:10  loss: 5.4711 (5.3798)  acc1: 1.5625 (4.9716)  acc5: 17.1875 (26.5625)  time: 0.1689  data: 0.1045  max mem: 4818
Test:  [ 20/782]  eta: 0:01:54  loss: 5.3996 (5.4018)  acc1: 4.6875 (7.5149)  acc5: 12.5000 (25.3720)  time: 0.1242  data: 0.0598  max mem: 4818
Test:  [ 30/782]  eta: 0:01:43  loss: 5.5631 (5.4864)  acc1: 3.1250 (5.8468)  acc5: 12.5000 (20.1109)  time: 0.1197  data: 0.0553  max mem: 4818
Test:  [ 40/782]  eta: 0:01:39  loss: 5.6514 (5.5178)  acc1: 1.5625 (5.2210)  acc5: 9.3750 (18.1021)  time: 0.1168  data: 0.0530  max mem: 4818
Test:  [ 50/782]  eta: 0:01:33  loss: 5.6118 (5.5455)  acc1: 0.0000 (4.4424)  acc5: 7.8125 (16.8199)  time: 0.1126  data: 0.0489  max mem: 4818
Test:  [ 60/782]  eta: 0:01:34  loss: 5.7426 (5.5502)  acc1: 0.0000 (4.2777)  acc5: 7.8125 (16.7777)  time: 0.1258  data: 0.0620  max mem: 4818
Test:  [ 70/782]  eta: 0:01:30  loss: 5.7426 (5.5908)  acc1: 1.5625 (4.2254)  acc5: 7.8125 (15.9111)  time: 0.1258  data: 0.0615  max

...

Test:  [620/782]  eta: 0:00:18  loss: 2.7866 (2.2207)  acc1: 37.5000 (51.6002)  acc5: 62.5000 (76.3084)  time: 0.1130  data: 0.0486  max mem: 4818
Test:  [630/782]  eta: 0:00:17  loss: 2.9873 (2.2268)  acc1: 37.5000 (51.5254)  acc5: 62.5000 (76.1737)  time: 0.1094  data: 0.0450  max mem: 4818
Test:  [640/782]  eta: 0:00:16  loss: 2.8601 (2.2344)  acc1: 37.5000 (51.3066)  acc5: 65.6250 (76.0262)  time: 0.1156  data: 0.0508  max mem: 4818
Test:  [650/782]  eta: 0:00:15  loss: 2.4787 (2.2387)  acc1: 35.9375 (51.2145)  acc5: 71.8750 (75.9673)  time: 0.1153  data: 0.0501  max mem: 4818
Test:  [660/782]  eta: 0:00:14  loss: 2.7760 (2.2552)  acc1: 32.8125 (50.9054)  acc5: 62.5000 (75.7139)  time: 0.1043  data: 0.0393  max mem: 4818
Test:  [670/782]  eta: 0:00:13  loss: 2.7760 (2.2582)  acc1: 40.6250 (50.8872)  acc5: 70.3125 (75.7009)  time: 0.1141  data: 0.0491  max mem: 4818
Test:  [680/782]  eta: 0:00:11  loss: 2.5197 (2.2678)  acc1: 39.0625 (50.6631)  acc5: 70.3125 (75.5415)  time: 0.1160 

Test:  [180/782]  eta: 0:01:14  loss: 1.7765 (1.8090)  acc1: 56.2500 (58.4254)  acc5: 82.8125 (83.3736)  time: 0.1211  data: 0.0512  max mem: 4818
Test:  [190/782]  eta: 0:01:12  loss: 1.9995 (1.8088)  acc1: 51.5625 (58.1152)  acc5: 84.3750 (83.5160)  time: 0.1108  data: 0.0407  max mem: 4818
Test:  [200/782]  eta: 0:01:11  loss: 1.7095 (1.8016)  acc1: 51.5625 (58.0846)  acc5: 87.5000 (83.6909)  time: 0.1087  data: 0.0387  max mem: 4818
Test:  [210/782]  eta: 0:01:09  loss: 1.7136 (1.8050)  acc1: 53.1250 (57.8940)  acc5: 84.3750 (83.7011)  time: 0.1154  data: 0.0464  max mem: 4818
Test:  [220/782]  eta: 0:01:09  loss: 1.8813 (1.8165)  acc1: 51.5625 (57.6004)  acc5: 81.2500 (83.5549)  time: 0.1407  data: 0.0715  max mem: 4818
Test:  [230/782]  eta: 0:01:08  loss: 1.4218 (1.7925)  acc1: 68.7500 (58.2725)  acc5: 87.5000 (83.8609)  time: 0.1323  data: 0.0628  max mem: 4818
Test:  [240/782]  eta: 0:01:06  loss: 1.4408 (1.7915)  acc1: 68.7500 (58.2923)  acc5: 87.5000 (83.8758)  time: 0.1158 

Test:  [740/782]  eta: 0:00:05  loss: 1.6602 (2.2006)  acc1: 56.2500 (51.7080)  acc5: 90.6250 (76.6405)  time: 0.1312  data: 0.0607  max mem: 4818
Test:  [750/782]  eta: 0:00:03  loss: 1.4251 (2.1913)  acc1: 68.7500 (51.9557)  acc5: 92.1875 (76.7893)  time: 0.1291  data: 0.0588  max mem: 4818
Test:  [760/782]  eta: 0:00:02  loss: 1.6262 (2.1951)  acc1: 53.1250 (51.8520)  acc5: 87.5000 (76.7432)  time: 0.1162  data: 0.0469  max mem: 4818
Test:  [770/782]  eta: 0:00:01  loss: 2.2200 (2.1874)  acc1: 51.5625 (52.0246)  acc5: 79.6875 (76.8746)  time: 0.1154  data: 0.0466  max mem: 4818
Test:  [780/782]  eta: 0:00:00  loss: 1.2425 (2.1781)  acc1: 71.8750 (52.2747)  acc5: 89.0625 (77.0146)  time: 0.1257  data: 0.0567  max mem: 4818
Test:  [781/782]  eta: 0:00:00  loss: 1.2714 (2.1816)  acc1: 65.6250 (52.2640)  acc5: 87.5000 (77.0020)  time: 0.1232  data: 0.0567  max mem: 4818
Test: Total time: 0:01:34 (0.1212 s / it)
* Acc@1 52.264 Acc@5 77.002 loss 2.182
Accuracy of the network on the 50000 

Test:  [300/782]  eta: 0:00:58  loss: 1.3428 (1.7064)  acc1: 67.1875 (60.5586)  acc5: 87.5000 (85.2004)  time: 0.1302  data: 0.0652  max mem: 4818
Test:  [310/782]  eta: 0:00:57  loss: 1.3428 (1.6979)  acc1: 71.8750 (60.8169)  acc5: 89.0625 (85.2693)  time: 0.1353  data: 0.0699  max mem: 4818
Test:  [320/782]  eta: 0:00:55  loss: 1.5193 (1.7085)  acc1: 60.9375 (60.6600)  acc5: 85.9375 (85.1246)  time: 0.1111  data: 0.0455  max mem: 4818
Test:  [330/782]  eta: 0:00:54  loss: 2.3810 (1.7415)  acc1: 48.4375 (60.0972)  acc5: 76.5625 (84.5591)  time: 0.1133  data: 0.0453  max mem: 4818
Test:  [340/782]  eta: 0:00:53  loss: 2.4151 (1.7559)  acc1: 46.8750 (59.8561)  acc5: 73.4375 (84.3383)  time: 0.1060  data: 0.0379  max mem: 4818
Test:  [350/782]  eta: 0:00:51  loss: 2.2962 (1.7795)  acc1: 50.0000 (59.4240)  acc5: 76.5625 (83.9209)  time: 0.1083  data: 0.0424  max mem: 4818
Test:  [360/782]  eta: 0:00:50  loss: 2.5543 (1.8022)  acc1: 42.1875 (58.9162)  acc5: 71.8750 (83.5570)  time: 0.1187 

KeyboardInterrupt: 